Primero instalamos las bibliotecas necesarias

In [1]:
!pip uninstall transformers
!pip uninstall accelerate
!pip uninstall bitsandbytes
!pip uninstall peft

!pip install "accelerate==0.29.3"
!pip install transformers
!pip install huggingface_hub transformers
!pip install "bitsandbytes==0.43.1"
!pip install peft

Found existing installation: transformers 4.46.2
Uninstalling transformers-4.46.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.46.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.46.2
Found existing installation: accelerate 1.1.1
Uninstalling accelerate-1.1.1:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-estimate-memory
    /usr/local/bin/accelerate-launch
    /usr/local/bin/accelerate-merge-weights
    /usr/local/lib/python3.10/dist-packages/accelerate-1.1.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/accelerate/*
Proceed (Y/n)? y
  Successfully uninstalled accelerate-1.1.1
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/peft-0.13.2.dist-info/*
    /usr/local/lib/python3.10/dist-pa

Realizamos los imports

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from huggingface_hub import notebook_login

Realizamos el login con nuestro token generado en huggingface (desmarcar la opción de git)

In [3]:
notebook_login()

Comprobamos si nos hemos registrado correctamente, si no sale nada es que lo  hemos hecho mal

Introducimos el modelo que queremos usar y lo configuramos con 4bits

In [4]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

Cargamos el modelo

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Introducimos el prompt y generamos la salida

In [7]:
prompt = (
    "Manten una conversación entre un señor que va a comprar una tarta para su cumpleaños y el de la pasteleria.\n"
    "Juan: Venia a por una tarta\n"
    "Pastelero:"
)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Manten una conversación entre un señor que va a comprar una tarta para su cumpleaños y el de la pasteleria.
Juan: Venia a por una tarta
Pastelero: ¡Bienvenido! ¿Qué deseas?
Juan: Una tarta para mi cumpleaños
Pastelero: ¿Qué tipo de tarta? ¿De chocolate, de frutas o de vainilla?
Juan: No sé, ¿qué te parece?
Pastelero: Bueno, nuestra tarta de chocolate es muy popular. También tenemos una tarta de frutas con frutas frescas y una tarta de vainilla con una textura suave.
Juan: Me gustaría probarla. ¿Puedo probar la de chocolate?
Pastelero: Claro, sí. (le sirve un trozo) ¿Te gusta?
Juan: ¡Sí! Me gusta mucho.
Pastelero: Me alegra saberlo. ¿Quieres comprarla?
Juan: Sí, ¿cómo es el costo?
Pastelero: La tarta de chocolate cuesta $5 y la tarta de frutas cuesta $6. La tarta de vainilla cuesta $7.
Juan: Me parece que la tarta de chocolate es la más económica. La compraré.
Pastelero: ¡Genial! (le hace la venta) ¿Quieres que la envuelva?
Juan
